In [4]:
import daft
import numpy as np

In [2]:
GLOB_PATH: str = "../../tests/test-data/butterflies_baseline/data/originals/*.jpg"

In [3]:
df_img = daft.from_glob_path(GLOB_PATH).with_column_renamed("path", "img_path")

# create image name column
df_img = df_img.with_column("img_name",
                   df_img["img_path"].str.split("/").list.get(
                       -1).cast(str))

# get the images since we'll need to use them several times
df_img = df_img.with_column("img", daft.col("img_path"
                                   ).url.download(on_error="null"
                                                  ).image.decode(on_error="null",
                                                                 mode="RGB")).collect()

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/daft/context.py:168: UserWarning: Daft is configured to use the new NativeRunner by default as of v0.4.0. If you are encountering any regressions, please switch back to the legacy PyRunner via `daft.context.set_runner_py()` or by setting the env variable `DAFT_RUNNER=py`. We appreciate you filing issues and helping make the NativeRunner better: https://github.com/Eventual-Inc/Daft/issues
  warnings.warn(
/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# | export

def autocontrast(img_arr: np.array) -> np.array:
    """autocontrast lifted from keras library
    added lightness normalization"""
    x = img_arr.astype(float)
    mean_before = x.mean()
    ## autocontrast from Keras
    x = x - np.min(x)
    x_max = np.max(x)
    if x_max != 0:
        x /= x_max
    x *= 255
    ## return to average lightness of input image
    mean_shift = x.mean() - mean_before
    x = np.clip(x - mean_shift, 0, 255)
    return x.astype("uint8")

In [8]:
@daft.udf(return_dtype=daft.DataType.image())
def autocontrast_img(arrs: daft.Series) -> np.array:
    """apply autocontrast to an image column"""
    imgs = arrs.to_pylist()
    return [autocontrast(img) for img in imgs]

In [9]:
df_img = df_img.with_column("img_autocontrast", autocontrast_img(df_img["img"]))

In [10]:
df_img.collect()

╭────────────────────────────────┬───────┬──────────┬────────────────────────────────┬────────────┬──────────────────╮
│ img_path                       ┆ size  ┆ num_rows ┆ img_name                       ┆ img        ┆ img_autocontrast │
│ ---                            ┆ ---   ┆ ---      ┆ ---                            ┆ ---        ┆ ---              │
│ Utf8                           ┆ Int64 ┆ Int64    ┆ Utf8                           ┆ Image[RGB] ┆ Image[MIXED]     │
╞════════════════════════════════╪═══════╪══════════╪════════════════════════════════╪════════════╪══════════════════╡
│ file://../../tests/test-data/… ┆ 42534 ┆ None     ┆ 0dae7d86-9c14-11ed-a00f-a37ce… ┆ <Image>    ┆ <Image>          │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ file://../../tests/test-data/… ┆ 40028 ┆ None     ┆ 30aeb051d-ee0d-4c5b-8a85-a8da… ┆ <Image>    ┆ <Image>          │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ file://../../tests/test-data/… ┆ 81735 ┆ None     ┆ 31fd87d81-7ff5-4e50-b718-0d4a… ┆ <Image>    ┆ <Image>          │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ file://../../tests/test-data/… ┆ 79596 ┆ None     ┆ 329a4c094-8536-4396-be70-3d9b… ┆ <Image>    ┆ <Image>          │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ file://../../tests/test-data/… ┆ 55851 ┆ None     ┆ 329c2b4d5-8137-414b-b98e-ff04… ┆ <Image>    ┆ <Image>          │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ file://../../tests/test-data/… ┆ 87727 ┆ None     ┆ 354747d21-638c-4e23-a655-c0bb… ┆ <Image>    ┆ <Image>          │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ file://../../tests/test-data/… ┆ 66581 ┆ None     ┆ 36829498c-0eda-4c84-b89d-c893… ┆ <Image>    ┆ <Image>          │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ file://../../tests/test-data/… ┆ 40259 ┆ None     ┆ 376dd3835-ea28-48ab-b7e8-c7c9… ┆ <Image>    ┆ <Image>          │
╰────────────────────────────────┴───────┴──────────┴────────────────────────────────┴────────────┴──────────────────╯

(Showing first 8 of 12 rows)